In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from keras.models import load_model

In [ ]:
import cobra
%load_ext autoreload
import sys
if not '/home/nlarusstone/cf_fba' in sys.path:
    sys.path.append('/home/nlarusstone/cf_fba')
import src.utils as utils
import src.flux_sample as fs
import src.create_dataset as dataset
from src.get_test_data import get_test_data
import src.cf_io as cf_io
import src.plotting as plotting
%autoreload 2

In [ ]:
latent_dim = 2
use_corr = True
froot = 'hand'
txtl = True
resamp = True
layer_szs = [1024, 1024, 1024]

In [ ]:
encoder, generator, X_test, y_test, obj_col, cols, y_vals_d, test_enc, test_dec = cf_io.get_test_data(
    froot, txtl, resamp, latent_dim, layer_szs, use_corr=True)

In [ ]:
latent_dim = 10
encoder_l, generator_l, X_test_l, y_test_l, obj_col_l, cols_l, y_vals_d_l, test_enc_l, test_dec_l = cf_io.get_test_data(
    froot, txtl, resamp, latent_dim, layer_szs, use_corr=True)

In [ ]:
df = cf_io.get_exp_data('manual')
df.drop('level_0', axis=1, inplace=True)
df.head()

In [ ]:
axarr = plt.subplots(nrows=1, ncols=2, figsize=(14, 8))[1]
plotting.plt_latent_space(test_enc, df, axarr[0], samp_range=(0, 1000), legend=False)
axarr[0].set_title('Corr-VAE latent space 2 dimensions')
plotting.plt_latent_space(test_enc_l, df, axarr[1], dim_1=0, dim_2=samp_range=(0, 1000), legend=True)
axarr[1].set_title('Corr-VAE latent space 10 dimensions')
plt.show()

In [ ]:
test_enc_l.shape

In [ ]:
from sklearn.manifold import TSNE
tsne = TSNE(n_components=2, verbose=True)

In [ ]:
test_enc_l.shape

In [ ]:
test_enc_sm = test_enc_l[0:1000, :, :]

In [ ]:
test_enc_l_2d = test_enc_sm.reshape(test_enc_sm.shape[0] * test_enc_l.shape[1], test_enc_l.shape[2])
test_enc_l_2d.shape

In [ ]:
tsne_enc_l = tsne.fit_transform(test_enc_l_2d)

In [ ]:
tsne_enc_l.shape

In [ ]:
col = np.array(range(17) * (tsne_enc_l.shape[0] / 17))

In [ ]:
plotting.plt_latent_space(tsne_enc_l, df, None, flat=True, y_test=col, dim_1=0, dim_2=1, legend=True)
plt.title('tSNE projection of 10-dimensional Corr-VAE latent space')

In [ ]:
for i in range(1, 10):
    plotting.plt_latent_space(test_enc_l, df, None, dim_1=0, dim_2=i, samp_range=(0, 1000), legend=True)
    plt.show()

In [ ]:
stds.shape

In [ ]:
stds = X_test.std(axis=0).mean(axis=0)
recon_stds = test_dec.std(axis=0).mean(axis=0)
rxn_nums = range(X_test.shape[-1])

In [ ]:
def plt_stds(rxns, stds, recon, ax=None, last=False):
    if ax is None:
        fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(10, 10))
    ax.scatter(rxns, stds)
    if not last:
        ax.set_ylabel('Std deviation of fluxes')
    ax.set_xlabel('Reaction ID')
    ax.set_title('Standard deviation of {0}fluxes'.format('reconstructed ' if recon else ''))
    #plt.show()
fig, axarr = plt.subplots(nrows=1, ncols=2, figsize=(10, 8))
plt_stds(rxn_nums, stds, recon=False, ax=axarr[0])
plt_stds(rxn_nums, recon_stds, recon=True, ax=axarr[1], last=True)
plt.show()

In [ ]:
def pred(biased_resamp_data, enc, gen):
    encoded_biased = enc.predict(biased_resamp_data)
    decoded_biased = gen.predict(encoded_biased)
    return check_corr(decoded_biased, df, btol_col)
pred(biased_resamp_data, enc, gen)
pred(biased_resamp_data, enc2, gen2)

In [ ]:
def add_noise(biased_resamp_data, enc, gen):
    noise_arr = np.logspace(start=-5, stop=1, num=10)
    corrs = []
    for noise in noise_arr:
        noisy_data = biased_resamp_data.copy()
        for i in range(n_experiments):
            s = noisy_data[:, i, :].shape
            noisy_data[:, i, :] += np.random.normal(scale=noise, size=s)
            #noisy_data[:, i, :] = minmax_scale(noisy_data[:, i, :])
        #scaled_noisy_data = scale_by_flux(noisy_data)
        corr = pred(noisy_data, enc, gen)
        corrs.append(corr)
    return zip(noise_arr, corrs) + [(0, pred(biased_resamp_data, enc, gen))]
noise_res = add_noise(biased_resamp_data, enc, gen)
noise_res2 = add_noise(biased_resamp_data, enc_good, gen_good)
noise_res3 = add_noise(biased_resamp_data, enc_bad, gen_bad)

In [ ]:
orig_corr = check_corr(biased_resamp_data, df, btol_col)
def plt_noise_corr(noise_data, orig_corr, ndim=2):
    orig_enc, noise_data = noise_data[-1], noise_data[:-1]
    plt.figure(figsize=(10, 8))
    plt.title('Latent dimension = {0}'.format(ndim))
    plt.axhline(y=-1 * orig_enc[1] if orig_enc[1] < 0 else orig_enc[1], label='Original data encoded')
    plt.axhline(y=orig_corr, label='Original data correlation', c='g')
    for noise, corr in noise_data:
        plt.scatter(x=noise, y=-1 * corr if corr < 0 else corr)
        plt.xlabel('Noise amount')
        plt.ylabel('Correlation')
    plt.xscale('log')
    plt.legend()
    plt.show()
plt_noise_corr(noise_res, orig_corr, ndim=2)
#plt_noise_corr(noise_res2, orig_corr, ndim=10)
#plt_noise_corr(noise_res3, orig_corr, ndim='2, no correlation loss')